# Data processing of proteomics

This only covers the models that are fit using proteomics data only

Various imports

In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import MultiTaskLasso
from sklearn.metrics import r2_score 
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.random_projection import SparseRandomProjection

Read the data with outliers

In [ ]:
def read_semiRaw_data():
    p100 = pd.read_csv('Final Data/p100_with_smiles_but_no_metadata.csv')
    jtvae_conv = pd.read_csv('Final Data/smiles_to_jtvae.csv')
    full_data = p100.merge(jtvae_conv, how='left',on='canonical_smiles')
    full_data.index = full_data['cid']
    full_data.drop('cid',axis=1,inplace=True)
read_semiRaw_data()

Read the clean data

In [ ]:
full_data = pd.read_csv('Final Data/janky_interlaced_full_data.csv',index_col='cid')

Outlier correction

In [ ]:
full_data=jank_outlier_correction(full_data)

In [ ]:
def jank_outlier_correction(full_data):
    for cell_id in list(dict.fromkeys(full_data['cell_id'])):                               #do it cell spesific
        print(cell_id)
        _ = full_data[full_data['cell_id']==cell_id]
        for canon in list(dict.fromkeys(_['canonical_smiles'])) :          #only work with non-duplicate canonicals
            tmp=_[_['canonical_smiles']==canon]
            stds = []
            for column in tmp.columns[3:-56]:                       #skip metadata and jtvae
                stds.append(np.std(tmp[column]))                    #get the standart deviation of a drug effect on 1 target
            _mean = np.mean(stds)
            for std in range(len(stds)):
                column = tmp.columns[3+std]
                if stds[std] > _mean:                               #if there are standart deviations that deviate more than others
                    _sub = []
                    for val in tmp[column]:                         #replace it's max value with the average of others
                        _sub = abs(val - tmp[column])
                    max_index = tmp[column].index[np.where(np.amax(_sub) == _sub)[0][0]]
                    full_data[column].loc[max_index] = np.mean(tmp[column].iloc[np.where(np.amax(_sub)!=_sub)])
    return full_data

In [ ]:
#jank_outlier_correction(full_data).to_csv('Final Data/janky_interlaced_full_data.csv') #write the cleaned data back

In [ ]:
def filter_metadata(data):
    return data.drop(['cell_id','pert_type','canonical_smiles'],axis=1)

In [ ]:
def get_feature_and_target(data):
    only_jtvae = pd.DataFrame()
    for column in data.columns:
        if column.startswith('jtvae'):
            only_jtvae = pd.concat((only_jtvae, data[column]), axis=1)
    return only_jtvae, data.drop(only_jtvae.columns, axis=1)

In [ ]:
def get_folds(compound_list, splits=5,random_state=666):
    compound_list = list(dict.fromkeys(compound_list))
    kf = KFold(shuffle=True,n_splits=splits,random_state=random_state)
    kf.get_n_splits(compound_list)

    fold_comp = []
    for train_index, test_index in kf.split(compound_list):
         fold_comp.append([np.asarray(compound_list)[train_index], np.asarray(compound_list)[test_index]])
    return fold_comp   #list that hold tuples of train and test compounds (in that order)

In [ ]:
def reduce_cols(train, test, target_reduction, tr_type, whiten=True):
    if tr_type == 'PCA':
        reducer = PCA(n_components=target_reduction, whiten=whiten).fit(train)
    elif tr_type == 'FastICA':
        reducer = FastICA(n_components=target_reduction, whiten=whiten).fit(train)
    elif tr_type == 'RandomProjection':
        reducer = SparseRandomProjection(n_components=target_reduction).fit(train)
    return reducer.transform(train), reducer.transform(test)

In [ ]:
def get_data_of_fold_i_for_gene(fold, full_data, cell, target_reduction=None, normalize=False, tr_type='PCA', whiten=True, n_splits=5, random_state=666):
    data = full_data[full_data['cell_id'] == cell]
    folds = get_folds(data['canonical_smiles'], splits=5, random_state=666)
    train = pd.DataFrame()
    test = pd.DataFrame()
    for canonical_smile in list(dict.fromkeys(data['canonical_smiles'])):
        if canonical_smile in folds[fold][0]:
            train = pd.concat((train,data[data['canonical_smiles']==canonical_smile]),axis=0)
        elif canonical_smile in folds[fold][1]:
            test = pd.concat((test,data[data['canonical_smiles']==canonical_smile]),axis=0)
        else:
            raise ValueError('The smiles doesn\'t exist in the folds.')
    
    train = filter_metadata(train)
    test = filter_metadata(test)

    X_train, y_train = get_feature_and_target(train)
    X_test, y_test = get_feature_and_target(test)
    
    if normalize:
        scaler = MinMaxScaler((0,100)).fit(y_train)
        y_train = scaler.transform(y_train)
        y_test = scaler.transform(y_test)
    if target_reduction is not None:
        y_train, y_test = reduce_cols(y_train, y_test, target_reduction, tr_type)
    
    
    return X_train, X_test, y_train, y_test

In [ ]:
import lightgbm as lgb

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

In [ ]:
def build_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(units=64, input_dim=input_dim))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=32))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=32))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=output_dim, activation='linear'))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    return model

In [ ]:
X_train, X_test, y_train, y_test = get_data_of_fold_i_for_gene(0, full_data, 'MCF7', target_reduction=1, tr_type='PCA')
# y_train  = np.asanyarray(y_train).ravel()
# y_test  = np.asanyarray(y_test).ravel()
ll = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.001, boosting_type='goss', reg_alpha=0.1, reg_lambda=0.)#, early_stopping_rounds=150, objective='regression')
# rgb = MultiOutputRegressor(ll)
rgb = build_model(X_train.shape[1], y_train.shape[1])
rgb.fit(X_train, y_train)#,eval_set=(X_test,y_test))
mean_absolute_error(y_test, rgb.predict(X_test))

In [ ]:
from keras.layers import Conv2D
from keras.optimizers import RMSprop
from sklearn.metrics import mean_absolute_error
import optuna
import mlflow.keras
import mlflow


In [ ]:
def objective(trial):
    X_train, X_test, y_train, y_test = get_data_of_fold_i_for_gene(0, full_data, cell, target_reduction=None, tr_type='FastICA')
    n_inputs = X_train.shape[1]
    n_outputs = y_train.shape[1]
    
    n_outputs = trial.suggest_int('target',1,n_outputs)
    model = Sequential()
    model.add(Dense(trial.suggest_int('hidden_size',1,200), input_dim=n_inputs, activation='relu'))
    model.add(Dropout(trial.suggest_uniform('dropout_1st',0.0,1.0)))
    model.add(Dense(trial.suggest_int('hidden_size2',0,200), activation='relu'))
    model.add(Dropout(trial.suggest_uniform('dropout_2nd',0.0,1.0)))
    model.add(Dense(n_outputs))
    model.add(Dropout(trial.suggest_uniform('dropout_3rd',0.0,1.0)))

    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    model.compile(loss='mae', optimizer=RMSprop(lr=lr))
    
    avg = []
    for i in range(5):
        X_train, X_test, y_train, y_test = get_data_of_fold_i_for_gene(i, full_data, 'MCF7', target_reduction=n_outputs, tr_type='FastICA', normalize=True)
        model.fit(X_train, y_train,epochs=trial.suggest_int('epochs',1,10))
        avg.append(mean_absolute_error(model.predict(X_test),y_test))
    return np.mean(avg)

In [ ]:
def build_exact_model(params):
    X_train, X_test, y_train, y_test = get_data_of_fold_i_for_gene(0, full_data, cell, target_reduction=None, tr_type='FastICA')
    n_inputs = X_train.shape[1]
    n_outputs = y_train.shape[1]
    
    n_outputs = params['target']
    model = Sequential()
    model.add(Dense(params['hidden_size'], input_dim=n_inputs, activation='relu'))
    model.add(Dropout(params['dropout_1st']))
    model.add(Dense(params['hidden_size2'], activation='relu'))
    model.add(Dropout(params['dropout_2nd']))
    model.add(Dense(n_outputs))
    model.add(Dropout(params['dropout_3rd']))

    lr = params['lr']
    model.compile(loss='mae', optimizer=RMSprop(lr=lr))
    
    X_train, X_test, y_train, y_test = get_data_of_fold_i_for_gene(0, full_data, cell, target_reduction=n_outputs, tr_type='FastICA', normalize=True)
    X = pd.concat((X_train,X_test),axis=0)
    y = pd.concat((pd.DataFrame(y_train),pd.DataFrame(y_test)),axis=0)
    model.fit(X, y,epochs=params['epochs'])
    return model

In [ ]:
for cell in ['A549', 'MCF7', 'PC3']:
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=250)
    mlflow.set_tracking_uri('http://tanlab.punoqun.cyou:5000')
    mlflow.set_experiment('P100 Experiments')
    mlflow.start_run()
    mlflow.set_tag('Author', 'Bahattin')
    mlflow.log_param('Model', 'NN')
    mlflow.log_param('cell_id', cell)
    mlflow.log_param('n_fold', '5')
    mlflow.log_param('random_state', '666')
    mlflow.log_param('optimizer', 'adam')
    mlflow.log_param('loss', 'mean_absolute_error')
    mlflow.log_param('target_reduction', 'FastICA')
    
    mlflow.log_param('mae', str(study.best_value))
    for param in study.best_params:
        mlflow.log_param(param, str(study.best_params[param]))
    build_exact_model(study.best_params)
    mlflow.keras.log_model(build_exact_model(study.best_params,cell))
#     ma.log_artifact(mlflow.get_artifact_uri())
    model.save(cell)
    mlflow.end_run()